In [12]:
# %cd ./filelists/CUB/
# !ln -s  ../../../CUB_200_2011/images ./images
# %cd ../../

# !ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./filelists/CUB/images

# !ln -s /kaggle/input/miniimagenet/miniImageNet/images ./filelists/miniImagenet/images
# !ln -s /kaggle/input/d/arjun2000ashok/tieredimagenet/tiered_imagenet/ ./filelists/tieredImagenet/images
# !ln -s /kaggle/input/stanford-dogs-dataset/images/Images ./filelists/dogs/Images
# !ln -s /kaggle/input/stanford-cars-dataset/cars_train/cars_train ./filelists/cars/images 
# !ln -s /kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data ./filelists/cars/images 
# !ln -s /kaggle/input/fgvc-aircraft/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/images ./filelists/aircrafts/images
# !ln -s /kaggle/input/vggflowers/images ./filelists/flowers/images

ln: failed to create symbolic link './filelists/flowers/images/images': Read-only file system


In [13]:
pwd

'/kaggle/working/fsl_ssl'

In [ ]:
!python train.py --dataset flowers --train_aug --method protonet --committed --stop_epoch 400 --jigsaw --lbda 0.5

Checkpoint path: ckpts/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_jigsaw_lbda0.50Adam_lr0.0010
Fresh wandb run
wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210608_075433-2otl6oqp
wandb: Syncing run good-sea-65
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/2otl6oqp
wandb: Run `wandb off` to turn off syncing.

About to start training. Last model and best model will be saved in wandb at every model save. 
 Run save_features.py and test.py after the training completes, with the same arguments
Epoch 0 | Batch 10/100 | Loss 12.797639 | Loss Proto 21.995384 | Loss Jigsaw 3.599893
Epoch 0 | Batch 20/100 | Loss 8.834058 | Loss Proto 14.097727 | Loss Jigsaw 3.5

In [1]:
# !python wandb_restore.py --id 14zlsjib --path ckpts/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/last_model.tar

wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210608_051230-14zlsjib
wandb: Resuming run flowers protonet
wandb: ⭐️ View project at https://wandb.ai/multi-input/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/multi-input/fsl_ssl/runs/14zlsjib
wandb: Run `wandb off` to turn off syncing.

Checkpoint restored at  ckpts/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/last_model.tar
The model's epoch is  319
Please rename it to continue training

wandb: Waiting for W&B process to finish, PID 837
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20210608_051230-14zlsjib/logs/debug.log
wandb: Find internal logs for t

In [5]:
# Do again with 200 dimensions
# Do again with 399.tar

!python save_features.py --dataset flowers --train_aug --method protonet --committed

Commit after saving features and testing.
checkpoint_dir: ckpts/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010
USE BN: True
outfile is features/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
0/31
10/31
20/31
30/31


In [6]:
!python test.py --dataset flowers --train_aug --method protonet --committed

Testing involves fine-tuning. Commit after testing.
novel_file features/flowers/_resnet18_protonet_aug_5way_5shot_16query_tracking_lr0.0010/novel.hdf5
600 Test Acc = 89.92% +- 0.51%
